In [1]:
"""
Python script to scrape Friktion User Data from Bitquery GraphQL API.

"""

import json
import requests
import requests
import traceback
import time

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from os.path import exists

In [79]:
globalId = "mainnet_income_put_luna"

users = pd.read_csv("luna_call_addresses.csv")

shareprice = pd.read_json(f"../derived_timeseries/{globalId}_sharePricesByGlobalId.json")

shareprice[0]/=1000

In [80]:
res = {}

In [81]:
def calc_amount(addr):
    success = False
    while not success:
        try:
            query = """https://solana-stream-dev-ztbl.ue1-eks-0.prod-czff.zettablock.dev/graphql?query={userTransactions(userAddress:\"%s\"){Hash,Amount,Decimal,InstructionAction,InstructionIndex,InnerIndex,UserAddress,Timestamp,UnixTime,CurrencyName,CurrencyAddress,SenderAddress,SenderTokenMint,ReceiverAddress,UserAction,GlobalID,VaultAuthority,ShareTokenMint,DepositTokenSymbol,DepositTokenCoingeckoId}}"""
            x = pd.DataFrame(json.loads(requests.get(query%addr).content)["userTransactions"])
            success = not x.empty
        except Exception as e:
            print("Failed... sleeping for 5")
            time.sleep(5)
            continue
            
    cutoff = 1651802400

    df = x.loc[x.UnixTime < cutoff].loc[x.GlobalID == globalId]

    df = df.sort_values("UnixTime")

    shareprice["timestamp"] = shareprice[0].astype('int')
    shareprice["sharePrice"] = shareprice[1]

    asimov = pd.merge_asof(df, shareprice, left_on="UnixTime", right_on="timestamp")

    pos = asimov.query("UserAction=='Deposit'")
    pos = (pos.Amount*pos.sharePrice/10**pos.Decimal).sum()

    pos2 = asimov.query("UserAction == 'CancelPendingWithdraw'")
    if pos2.empty:
        pos2 = 0
    else:
        pos2 = (pos2.Amount/10**pos2.Decimal).sum()

    neg = asimov.query("UserAction=='Withdraw'")
    if neg.empty:
        neg = 0
    else:
        neg = (neg.Amount/10**neg.Decimal).sum()

    neg2 = asimov.query("UserAction=='CancelPendingDeposit'")
    if neg2.empty:
        neg2 = 0
    else:
        neg2= (neg2.Amount*neg2.sharePrice/10**neg2.Decimal).sum()

    amount = pos+pos2-neg-neg2
    print(addr, amount)
    res[addr] = amount

In [66]:
for addr in users.userAddress:
    
    calc_amount(addr)
    time.sleep(1)

CQikhhQ9STuD1S3A45DGvUAtCdSs5pw2YwJ8Mk6U9Fs7 195.97417776146025
ETv2idiPjun5RReJNSLBd2RUMcYSBMqjFQxhVpLeDqkm 11.084006874905258
GZYHrKmMPQLrmVGw7PRny7tP9wQZnFCC5vYfuaQiVTdh 4.59754700000002


KeyboardInterrupt: 

In [88]:
    addr = "Dnz2w5wxrTRVyPvYZgAuEPBkV5kDFUeCFy1XieCJrBuE"
    query = """https://solana-stream-dev-ztbl.ue1-eks-0.prod-czff.zettablock.dev/graphql?query={userTransactions(userAddress:\"%s\"){Hash,Amount,Decimal,InstructionAction,InstructionIndex,InnerIndex,UserAddress,Timestamp,UnixTime,CurrencyName,CurrencyAddress,SenderAddress,SenderTokenMint,ReceiverAddress,UserAction,GlobalID,VaultAuthority,ShareTokenMint,DepositTokenSymbol,DepositTokenCoingeckoId}}"""

    x = pd.DataFrame(json.loads(requests.get(query%addr).content)["userTransactions"])

    cutoff = 165180960000

    df = x.loc[x.UnixTime < cutoff].loc[x.GlobalID == globalId]

    df = df.sort_values("UnixTime")

    shareprice["timestamp"] = shareprice[0].astype('int')
    shareprice["sharePrice"] = shareprice[1]

    asimov = pd.merge_asof(df, shareprice,  left_on="UnixTime", right_on="timestamp", direction='forward')
    asimov = asimov.sort_values("timestamp").drop_duplicates(subset = ['Hash', 'UserAction'], keep='first')
    pos = asimov.query("UserAction=='Deposit'")
    pos = (pos.Amount/pos.sharePrice/10**pos.Decimal).sum()

    pos2 = asimov.query("UserAction == 'CancelPendingWithdrawal'")
    if pos2.empty:
        pos2 = 0
    else:
        pos2 = (pos2.Amount/10**pos2.Decimal).sum()

    neg = asimov.query("UserAction=='Withdraw'")
    if neg.empty:
        neg = 0
    else:
        neg = (neg.Amount/10**neg.Decimal).sum()

    neg2 = asimov.query("UserAction=='CancelPendingDeposit'")
    if neg2.empty:
        neg2 = 0
    else:
        neg2= (neg2.Amount*neg2.sharePrice/10**neg2.Decimal).sum()

    amount = pos+pos2-neg-neg2
    
    res[addr] = amount

In [89]:
df

,Amount,CurrencyAddress,CurrencyName,Decimal,DepositTokenCoingeckoId,DepositTokenSymbol,GlobalID,Hash,InnerIndex,InstructionAction,InstructionIndex,ReceiverAddress,SenderAddress,SenderTokenMint,ShareTokenMint,Timestamp,UnixTime,UserAction,UserAddress,VaultAuthority
13,20004106,9vMJfxuKxXBoEa7rM12mYLMwTacLMLDJqHozw96WQL8i,UST (Wormhole),6,terrausd,UST,mainnet_income_put_luna,ai9gLTTwH1rkhhbYkM6XHaBhb84VTotE1yT9dAxFu7AB1B...,1,Transfer,1,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj,Dnz2w5wxrTRVyPvYZgAuEPBkV5kDFUeCFy1XieCJrBuE,4z8brRdRougR1Lkyfd49noxMYGE67Vbw4pW9eqQePz6H,74ozhToAS71nDVBtFZMMucdkNB95tV2o5fhtFGQeixwS,2022-03-31T18:03:16Z,1648749796,Deposit,Dnz2w5wxrTRVyPvYZgAuEPBkV5kDFUeCFy1XieCJrBuE,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj
6,2300000000000,9vMJfxuKxXBoEa7rM12mYLMwTacLMLDJqHozw96WQL8i,UST (Wormhole),6,terrausd,UST,mainnet_income_put_luna,3Pu7r5sJs75nr6VPmAo8BAFt7bZcYkp9jAcYo1Sd7s5oWb...,0,Transfer,1,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj,Dnz2w5wxrTRVyPvYZgAuEPBkV5kDFUeCFy1XieCJrBuE,4z8brRdRougR1Lkyfd49noxMYGE67Vbw4pW9eqQePz6H,74ozhToAS71nDVBtFZMMucdkNB95tV2o5fhtFGQeixwS,2022-04-04T20:46:01Z,1649105161,Deposit,Dnz2w5wxrTRVyPvYZgAuEPBkV5kDFUeCFy1XieCJrBuE,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj


In [78]:
asimov

,Amount,CurrencyAddress,CurrencyName,Decimal,DepositTokenCoingeckoId,DepositTokenSymbol,GlobalID,Hash,InnerIndex,InstructionAction,...,ShareTokenMint,Timestamp,UnixTime,UserAction,UserAddress,VaultAuthority,0,1,timestamp,sharePrice
0,1000000,F6v4wfAdJB8D8p77bMXZgYt8TDKsYxLYxH5AFhUkYx9W,LUNA (Wormhole),6,terra-luna,LUNA,mainnet_income_call_luna,59PPj5aoq12bxZrWC7xBHHNmw66o2aDiH2g2o1KoDG26Lb...,1,Transfer,...,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-02-04T18:14:09Z,1643998449,Deposit,HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S,5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X,1.644547e+09,1.056219,1644546713,1.056219
1,7000000,F6v4wfAdJB8D8p77bMXZgYt8TDKsYxLYxH5AFhUkYx9W,LUNA (Wormhole),6,terra-luna,LUNA,mainnet_income_call_luna,4Kbe3pEGz3gpR3whyjoBP7sktEd5cAf9UraLbNGUZd3WE9...,0,Transfer,...,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-02-04T18:32:15Z,1643999535,Deposit,HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S,5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X,1.644547e+09,1.056219,1644546713,1.056219
2,1098491,F6v4wfAdJB8D8p77bMXZgYt8TDKsYxLYxH5AFhUkYx9W,LUNA (Wormhole),6,terra-luna,LUNA,mainnet_income_call_luna,3p6PgCeuKUg4WnTqQRmuQhBg974Hqwme9nGuCteJNdqVjF...,0,Transfer,...,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-02-13T03:45:15Z,1644723915,Deposit,HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S,5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X,1.645152e+09,1.068417,1645152424,1.068417
3,992881,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,,6,terra-luna,LUNA,mainnet_income_call_luna,UNyr2X8WuBF3sjnsRQjXrjaidZnVefbnvCbeVPMcN51Mbn...,1,Burn,...,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-03-17T15:32:46Z,1647531166,Withdraw,HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S,5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X,1.647572e+09,1.010797,1647572109,1.010797
4,1002597,F6v4wfAdJB8D8p77bMXZgYt8TDKsYxLYxH5AFhUkYx9W,LUNA (Wormhole),6,terra-luna,LUNA,mainnet_income_call_luna,3nQVXWanrC9FhjpgUKxdZ7BGkphNme2hzSphfs3RYim1WW...,0,Transfer,...,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-03-23T12:59:20Z,1648040360,ClaimPendingWithdrawal,HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S,5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X,1.648180e+09,1.016702,1648179512,1.016702
5,304384,F6v4wfAdJB8D8p77bMXZgYt8TDKsYxLYxH5AFhUkYx9W,LUNA (Wormhole),6,terra-luna,LUNA,mainnet_income_call_luna,2SaUaTQ4L5CBoB3oSUaXtWjrLepXFrbP8M6Cea3CHkZHif...,0,Transfer,...,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-03-23T13:12:17Z,1648041137,Deposit,HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S,5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X,1.648180e+09,1.016702,1648179512,1.016702
6,7908837,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,,6,terra-luna,LUNA,mainnet_income_call_luna,2uEKPpivnvAavdsLvfT66wNUFV393YsUc62sT3WczYkxf1...,0,Burn,...,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-03-31T01:58:37Z,1648691917,Withdraw,HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S,5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X,1.648788e+09,1.019346,1648788461,1.019346


In [50]:
asimov.sort_values("timestamp").drop_duplicates(subset = ['Hash', 'UserAction'], keep='first')[["timestamp", "UnixTime"]].fillna(0).astype('int')

,timestamp,UnixTime
0,1640931286,0


In [44]:
asimov.columns

Index([                        0,                         1,
                     'timestamp',              'sharePrice',
                        'Amount',         'CurrencyAddress',
                  'CurrencyName',                 'Decimal',
       'DepositTokenCoingeckoId',      'DepositTokenSymbol',
                      'GlobalID',                    'Hash',
                    'InnerIndex',       'InstructionAction',
              'InstructionIndex',         'ReceiverAddress',
                 'SenderAddress',         'SenderTokenMint',
                'ShareTokenMint',               'Timestamp',
                      'UnixTime',              'UserAction',
                   'UserAddress',          'VaultAuthority'],
      dtype='object')

In [20]:
amount

-208.58390497893265

In [21]:
df.Amount*

SyntaxError: invalid syntax (3961119424.py, line 1)

In [25]:
pos = asimov.query("UserAction=='Deposit'")

In [26]:
(pos.Amount/pos.sharePrice.fillna(0)/10**pos.Decimal).sum()

128.45520002106738

In [28]:
10**6

1000000

In [30]:
pos.Amount/1000000

0    210.209253
1      0.092372
2     66.613782
4     40.000000
5     23.000000
Name: Amount, dtype: float64

In [31]:
pos.sharePrice

0         NaN
1         NaN
2    1.000000
4    1.018735
5    1.018735
Name: sharePrice, dtype: float64

In [17]:
pos2

61.348739

In [15]:
neg

398.38784400000003

In [9]:
neg2

0